In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
gender = pd.read_csv("gender_submission.csv")

In [171]:
# contar a quantidade de elemento não nulos, caso tenha muitos elementos nulos a coluna deve ser deletada
for coluna in train.columns:
    print(coluna)
    print("Quantidade de elementos não nulos: {}".format(train[coluna].count()))
    print("--------------------------------------")

PassengerId
Quantidade de elementos não nulos: 891
--------------------------------------
Survived
Quantidade de elementos não nulos: 891
--------------------------------------
Pclass
Quantidade de elementos não nulos: 891
--------------------------------------
Name
Quantidade de elementos não nulos: 891
--------------------------------------
Sex
Quantidade de elementos não nulos: 891
--------------------------------------
Age
Quantidade de elementos não nulos: 714
--------------------------------------
SibSp
Quantidade de elementos não nulos: 891
--------------------------------------
Parch
Quantidade de elementos não nulos: 891
--------------------------------------
Ticket
Quantidade de elementos não nulos: 891
--------------------------------------
Fare
Quantidade de elementos não nulos: 891
--------------------------------------
Cabin
Quantidade de elementos não nulos: 204
--------------------------------------
Embarked
Quantidade de elementos não nulos: 889
-----------------------

In [115]:
train["PassengerId"].describe() 

count    891.000000
mean     446.000000
std      257.353842
min        1.000000
25%      223.500000
50%      446.000000
75%      668.500000
max      891.000000
Name: PassengerId, dtype: float64

In [172]:
train["Survived"].describe() 

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [173]:
train["Pclass"].describe()

count    891.000000
mean       2.308642
std        0.836071
min        1.000000
25%        2.000000
50%        3.000000
75%        3.000000
max        3.000000
Name: Pclass, dtype: float64

In [174]:
train["Name"].describe() 

count                                891
unique                               891
top       Sobey, Mr. Samuel James Hayden
freq                                   1
Name: Name, dtype: object

In [175]:
train["Sex"].describe() 

count      891
unique       2
top       male
freq       577
Name: Sex, dtype: object

In [74]:
train["Age"].describe() 

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [80]:
train["SibSp"].describe() 

count    891.000000
mean       0.523008
std        1.102743
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        8.000000
Name: SibSp, dtype: float64

In [72]:
train["Parch"].describe() 

count    891.000000
mean       0.381594
std        0.806057
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: Parch, dtype: float64

In [71]:
train["Ticket"].describe() 

count      891
unique     681
top       1601
freq         7
Name: Ticket, dtype: object

In [70]:
train["Fare"].describe() 

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [69]:
train["Cabin"].describe() 

count     204
unique    147
top        G6
freq        4
Name: Cabin, dtype: object

In [68]:
# esta coluna contém apenas 3 possíveis valores 'C', 'Q', 'S'
train["Embarked"].describe() 

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [11]:
def preprocess_data(data):
    newdf = pd.DataFrame()

    newdf = pd.DataFrame()
    newdf['Age'] = data.Age.fillna(data.Age.mean())
    newdf['Sex'] = pd.Series([1 if s == 'male' else 0 for s in data.Sex], name = 'Sex')
    #newdf['Family'] = data['SibSp'] + data['Parch']
    #newdf['Survived'] = data['Survived']
    #newdf['Fare'] = data.Fare.fillna(data.Age.mean())
    newdf['Embarked'] = data.Embarked.fillna('S').map({'S':1,'C':2, 'Q':3})    

    pclass = pd.get_dummies(data.Pclass, prefix='Pclass')
    for c in pclass:
        newdf[c] = pclass[c]
    return newdf

In [9]:
train_data

,Age,Sex,Embarked,Pclass_1,Pclass_2,Pclass_3
0,22.000000,1,1,0,0,1
1,38.000000,0,2,1,0,0
2,26.000000,0,1,0,0,1
3,35.000000,0,1,1,0,0
4,35.000000,1,1,0,0,1
5,29.699118,1,3,0,0,1
6,54.000000,1,1,1,0,0
7,2.000000,1,1,0,0,1
8,27.000000,0,1,0,0,1
9,14.000000,0,2,0,1,0


In [8]:
train_data = preprocess_data(train)
train_labels = train.Survived

In [250]:
modelo = GaussianNB()
modelo.fit(train_data, train_labels)
test_data = preprocess_data(test)
predicao = modelo.predict(test_data)
print('Score: {}'.format(modelo.score(test_data, gender.Survived)))

Score: 0.8636363636363636


In [251]:
test_data = preprocess_data(test)
prediction = modelo.predict(test_data)

submission = pd.DataFrame()
submission['PassengerId'] = test.PassengerId
submission['Survived'] = pd.Series(prediction)
submission.to_csv("kaggle.csv", index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
